<h1><center>Peer-Eval Week3 Assignment </center></h1>

<h2><center>Srivatsan Mohan 30/7/21</center></h2>

### Setting up Environment 

In [ ]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup
from urllib.request import urlopen
print('Libraries imported.')

### Parsing html

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urlopen(url).read().decode('utf-8')
soup = BeautifulSoup(page, 'html.parser')


### Extracting data to dataframe

In [ ]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned': # cleaning data by dropping rows with bourroghs not assigned
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.head()

### Adding coordinates to table

In [ ]:
# No module named 'geocoder' so using csv file
# import geocoder
# def get_latlng(postal_code):
#     # initialize your variable to None
#     lat_lng_coords = None
#     # loop until you get the coordinates
#     while(lat_lng_coords is None):
#         g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
#         lat_lng_coords = g.latlng
#     return lat_lng_coords
# postal_codes = df['PostalCode']    
# coords = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]
# df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
# df['Latitude'] = df_coords['Latitude']
# df['Longitude'] = df_coords['Longitude']
# df.head()

#Sorting values by postalcode before adding to main dataframe
latlong=pd.read_csv("Geospatial_Coordinates.csv")
latlong = latlong.set_index('Postal Code')
latlong = latlong.reindex(index=df['PostalCode'])
latlong = latlong.reset_index()
df['Latitude'] = latlong['Latitude']
df['Longitude'] = latlong['Longitude']
df.head()


### Explore and cluster the neighborhoods in Toronto


In [57]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df .shape[0]
    )
)


The dataframe has 15 boroughs and 103 neighborhoods.


In [60]:
# set number of clusters

lat_long = np.stack((df['Latitude'],df['Longitude']), axis=1)
kmeans = KMeans(n_clusters=5, random_state=0).fit(lat_long)
clusters = kmeans.labels_
df['Cluster'] = clusters
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster
0,M3A,North York,Parkwoods,43.753259,-79.329656,4
1,M4A,North York,Victoria Village,43.725882,-79.315572,4
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,2


### Map Results

In [61]:
# create map
toronto_map = folium.Map(location=[43.65, -79.4], zoom_start=12)
# set color scheme for the clusters
colors = ['red', 'green', 'blue', 'yellow','black']
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'],df['Longitude'], df['Neighborhood'], df['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(toronto_map)
       
toronto_map